In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import os
import numpy as np

`metrics` should be included in `MRR`, `log`, `sqrt`, `P1_4`, `P1_3`, `P2_3`, `P3_4`, `HITS1`, `HITS3`, `HITS10`, where `PN_M` means $r^{N/M}$.

`prefix` should be `logs` if independent or `logs_cor` if correlated.

The generated figure is saved in dir `.\fig`.

In [ ]:
log_data = dict()
log_index = dict()
prefix = 'logs'
metrics = 'MRR'
sparse_list = [int(x) for x in filter(str.isdigit, os.listdir(prefix))]
for sparse in sparse_list:
    log_data[sparse] = dict()
    log_index[sparse] = dict()
    index = 0
    path = os.path.join(prefix, str(sparse))
    for current_path, dirlist, filelist in os.walk(path):
        dirlist = list(filter(lambda x: x[0]!='.', dirlist))
        if dirlist or not filelist:
            continue
        elif current_path.split('/')[-1][0] == '.':
            # ignore all hidden file
            continue
        else:
            log_index[sparse][current_path] = index
            event_file_list = filter(lambda x: 'events.out.tfevents' in x, os.listdir(current_path))
            ea_list = []
            for f in event_file_list:
                ea=event_accumulator.EventAccumulator(os.path.join(current_path, f))
                ea.Reload()
                ea_list.append(ea)
        log_data[sparse][index] = dict()
        full = []
        step = []
        sparsel = []
        for ea in ea_list:
            if not ea.scalars.Keys():
                continue
            length = min(len(ea.scalars.Items('Test_1p_{}'.format(metrics))), len(ea.scalars.Items('Test_1p_{}'.format(metrics))), len(ea.scalars.Items('Test_sparse_1p_{}'.format(metrics))))
            full += [x.value for x in ea.scalars.Items('Test_1p_{}'.format(metrics))][0:length]
            step += [x.step for x in ea.scalars.Items('Test_1p_{}'.format(metrics))][0:length]
            sparsel += [x.value for x in ea.scalars.Items('Test_sparse_1p_{}'.format(metrics))][0:length]
        log_data[sparse][index]['full'] = full
        log_data[sparse][index]['step'] = step
        log_data[sparse][index]['sparse'] = sparsel
        index += 1
print(log_index)

In [ ]:
colors_list = ['#191970', '#0000CD', '#4169E1', '#6495ED', '#00C0C0',
               '#8B008B', '#C000C0', '#C71585', '#9400D3', '#EE82EE',
               '#90EE90', '#ADFF2F', '#FFD700', '#C0C000', '#808000',
               '#8B0000', '#A0522D', '#F08080',
              ]

In [ ]:
for compare in sparse_list:
    max_range = 0
    print(compare)
    for path in log_index[compare]:
        if log_index[compare][path] in [16]:
            continue
        full = log_data[compare][log_index[compare][path]]['full']
        if not full:
            continue
        # max_range = max(max_range, max(sparse))
        sparse = log_data[compare][log_index[compare][path]]['sparse']
        max_range = max(max_range, max(sparse))
        length = min(len(full), len(sparse))
        argsort = np.argsort(np.array(full[:length]))
        plt.plot(np.array(full[:length])[argsort], np.array(sparse[:length])[argsort], label=str(log_index[compare][path]), c=colors_list[log_index[compare][path]], linewidth=0.7)
    # plt.plot(np.arange(0, max_range+0.1, 0.1), np.arange(0, max_range+0.1, 0.1), label='y=x', c='k')
    # plt.plot(np.arange(0, max_range+0.1, 0.1), 0.107*np.ones(np.arange(0, max_range+0.1, 0.1).shape[0]))
    plt.xlabel('full'.format(metrics), fontsize=15)
    plt.ylabel('sparse'.format(metrics), fontsize=15)
    #plt.legend()
    plt.title('{}'.format(metrics), fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.ylim(0,max_range+0.02)
    plt.savefig('fig/{}_sparse{}.pdf'.format(metrics, compare), bbox_inches='tight')
    # if need the legend, please change the linewidth = 5
    # lleeggeenndd = plt.legend(loc='center left', bbox_to_anchor=(10, 10), framealpha=1, frameon=False, handlelength=0.1)
    plt.show()